In [1]:
import glob, json, os, sys
sys.path.append('/root/SWE-bench')
sys.path.append('/root/SWE-bench/swebench/metrics') # TODO: Replace with path to `SWE-bench/swebench/metrics` folder
from conversion import convert_log_to_ground_truth
from getters import get_logs_gold
from monitor import monitor_validation, monitor_logs_same_diff
sys.path = sys.path[:-1]

sys.path.append('/root/SWE-bench/swebench/harness') # TODO: Replace with path to `SWE-bench/swebench/harness` folder
from utils import has_attribute_or_import_error
sys.path = sys.path[:-1]

/opt/conda/envs/swe-bench/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Declare repository; Fetch tasks, logs

In [47]:
# import glob 

# logs_list = glob.glob("/mnt/llm/home/ibragim-bad/data/swe_large/log_dir/*.log") + glob.glob("/mnt/llm/home/ibragim-bad/data/swe_large_new/log_dir/*.json")
# inst = [l.split('/')[-1].split('.')[0] for l in  logs_list]
# with open("/root/SWE-bench/swe_like_111k.json") as f:
#     tasks = json.load(f)
# cut_tasks = [t for t in tasks if t['instance_id'] not in set(inst)]

In [49]:
# from collections import defaultdict

# ct_dict = defaultdict(list)

# for t in cut_tasks:
#     ct_dict[t['repo']].append(t)

# mx_len = max([len(v) for v in ct_dict.values()])
# new_ct = []
# for i in range(mx_len):
#     for k in ct_dict:
#         if i < len(ct_dict[k]):
#             t = ct_dict[k][i]
#             new_ct.append(t)

In [54]:
# N = 500



# for i in range(0, len(new_ct), N):
#     with open(f"/root/data/swe/tasks_{i//N}.json", "w") as f:
#         json.dump(new_ct[i:i+N], f)

In [14]:
lst = open("/root/data/progress.txt").readlines()
lst = set([l.strip() for l in lst])

from collections import defaultdict

tasks = []
tasks_dict = defaultdict(set)
for l in lst:
    with open(f"/root/data/swe/{l}") as f:
        cur_t =  json.load(f)
        tasks +=cur_t
        for t in cur_t:
            tasks_dict[l.split('/')[0]].add(t['instance_id'])

In [64]:
# import json
# l = {'swebench': {'resolved': {'count': 7, 'turns': {'avg': 9.571428571428571, 'median': 9.0, 'p90': 13.4}, 'opened_files': {'avg_precision': 0.8571428571428571, 'avg_recall': 1.0, 'avg_jaccard_similarity': 0.8571428571428571}, 'edit_loop_fraction': 0.14285714285714285, 'exit_cost_fraction': 0.0, 'patched_files': {'avg_precision': 0.9285714285714286, 'avg_recall': 1.0, 'avg_jaccard_similarity': 0.9285714285714286}}, 'failed': {'count': 88, 'turns': {'avg': 22.545454545454547, 'median': 16.5, 'p90': 45.3}, 'opened_files': {'avg_precision': 0.3901515151515151, 'avg_recall': 0.4431818181818182, 'avg_jaccard_similarity': 0.3901515151515151}, 'edit_loop_fraction': 0.25, 'exit_cost_fraction': 0.0, 'patched_files': {'avg_precision': 0.369047619047619, 'avg_recall': 0.45454545454545453, 'avg_jaccard_similarity': 0.369047619047619}}, 'skipped': {'count': 191, 'turns': {'avg': 39.02094240837696, 'median': 30.0, 'p90': 82.0}, 'opened_files': {'avg_precision': 0.22168638516806055, 'avg_recall': 0.31413612565445026, 'avg_jaccard_similarity': 0.22168638516806055}, 'edit_loop_fraction': 0.35602094240837695, 'exit_cost_fraction': 0.0}, 'ignored': {'count': 6}, 'total': {'count': 292}}}
# print(json.dumps(l, indent=4))

In [13]:
repo = 'pydicom/pydicom' # TODO: Replace with repository name
log_dir = '/root/data/log_dir_2' # TODO: Replace with path to folder of execution logs

Get map of version to setup commit

In [15]:
# tasks = json.load(open('path/to/pydicom-task-instances.json')) # TODO: Replace with path to versioned candidate task instances
for t in tasks:
    t["version"] = "0.0"


tasks = sorted(tasks, key=lambda x: x['created_at'], reverse=True)

version_to_setup_commit = {}
for t in tasks:
    if 'version' in t and t['version'] not in version_to_setup_commit:
        version_to_setup_commit[t['version']] = t['base_commit']
assert(
    sorted(list([x or "" for x in version_to_setup_commit.keys()])) ==
    sorted(list(set([t['version'] or "" for t in tasks if 'version' in t])))
)
tasks = {t['instance_id']: t for t in tasks}

#### Monitor Validation

In [16]:
failed_install, corrupt_test_patch, corrupt_patch, timeout, success = monitor_validation(log_dir)

Total Attempts: 22687
Failed: 14862
Usable: 7825
Corrupt Test: 138
Corrupt Diff: 80
Test Script Timeout: 64
Success E2E: 7543


In [17]:
len([f for f in failed_install if '0.0' not in f]) + len(success)

13682

In [60]:
# import os 

# fails = set([os.path.basename(t).split('.')[0] for t in failed_install if '0.0' not in t])

# for t in sorted(tasks_dict):
#     print(t, len(tasks_dict[t]), len(tasks_dict[t] & fails))

# tasks_dict['tasks_0.json'] & fails


In [18]:
logs_same, logs_diff = monitor_logs_same_diff("/root/data/swe_large/log_dir")
print(f"Logs same: {len(logs_same)}")
print(f"Logs diff: {len(logs_diff)}")

Logs same: 19909
Logs diff: 3082


In [90]:
len(tasks)

2500

#### Get [FP]2[FP] Tests

In [19]:
inst_id_to_gt = {}
for d in logs_diff:
    status_gt = convert_log_to_ground_truth(d[0])
    inst_id_to_gt[d[0]] = status_gt

#### Create Task Instances `.json` file

In [20]:
tasks_final = []
get_id_from_log = lambda x: x.split('/')[-1].split('.')[0]
for k, v in inst_id_to_gt.items():
    if len(v['FAIL_TO_PASS']) == 0:
        continue
    if not get_id_from_log(k) in tasks:
        continue
    task = tasks[get_id_from_log(k)]
    task['FAIL_TO_PASS'] = v['FAIL_TO_PASS']
    task['PASS_TO_PASS'] = v['PASS_TO_PASS']
    task['environment_setup_commit'] = version_to_setup_commit["0.0"]

    # Do not consider tasks where the log before the patch has an attribute/import error
    log_path = os.path.join(log_dir, f'{task["instance_id"]}.log')
    log_before, log_after = get_logs_gold(log_path)
    if has_attribute_or_import_error(log_before):
        continue

    tasks_final.append(task)
print(f"Final number of task instances: ", len(tasks_final))

# SAVE_PATH = "path/to/save/tasks/to.json" # TODO: Replace this with a path to a .json file to save the task instances
# with open(SAVE_PATH, 'w') as f:
#     json.dump(tasks_final, fp=f)

Final number of task instances:  0
